In [ ]:
import os
from tqdm import tqdm

import cv2
import numpy as np
import matplotlib.pyplot as plt

from utils.dataloader import DataLoader
from utils.vis import MatplotlibVisualizer
from utils.transforms import HairRemoval, Composer
from utils.utils import export_experiment
from descriptors.shape import HOGDescriptor
from utils.segmentation import ThresholdingSegmentation
from descriptors.stats import IntensityStatsGridDescriptor
from descriptors.texture import LBPDescriptor, GLCMDescriptor, GaborFilterDescriptor
from descriptors.color import ColorDescriptor, ColorLayoutDescriptor, ColorCooccurrenceMatrixDescriptor

In [ ]:
## Classes
CLASSES = ['mel', 'bcc', 'scc']

## Work folfer
work_folder = os.getcwd()
data_folder = os.path.join(work_folder, '..', 'Data/Challenge2')

## Visualizer
matplotlib_visualizer = MatplotlibVisualizer()
exp_name = 'multi-class_classification'

In [ ]:
transforms_composer = Composer([
                                HairRemoval(),
                                ])

## Featrure Extraction

### Descriptors

In [ ]:
## Define parameters
params = {
    'color_layout': {
        'grid_x': 3,
        'grid_y': 3,
    },
    'intensity_stats': {
        'grid_x': 3,
        'grid_y': 3,
    },
    'color': {
        'bins': (8, 12, 3),
        'grid_x': 1,
        'grid_y': 1,
    },
    'glcm': {
        'distances': [1],
        'angles': [0, np.pi/4, np.pi/2, 3*np.pi/4],
        'levels': 8,
        'grid_x': 3,
        'grid_y': 3,
    },
    'lbp': {
        'radius': 2,
        'n_points': 8,
        'grid_x': 3,
        'grid_y': 3,
    },
    'lbp2': {
        'radius': 3,
        'n_points': 16,
        'grid_x': 3,
        'grid_y': 3,
    },
    'lbp3': {
        'radius': 3,
        'n_points': 24,
        'grid_x': 3,
        'grid_y': 3,
    },
}

In [ ]:
modes = ['train', 'val']

## Descriptors
color_layout_descriptor = ColorLayoutDescriptor(**params['color_layout'])
intensity_stats_grid_descriptor = IntensityStatsGridDescriptor(**params['intensity_stats'])
color_descriptor = ColorDescriptor(**params['color'])
glcm_descriptor = GLCMDescriptor(**params['glcm'])
lbp_descriptor = LBPDescriptor(**params['lbp'])
lbp2_descriptor = LBPDescriptor(**params['lbp2'])
lbp3_descriptor = LBPDescriptor(**params['lbp3'])
# color_cooccurrence_matrix_descriptor = ColorCooccurrenceMatrixDescriptor(distances=[1], angles=[0, np.pi/4, np.pi/2, 3*np.pi/4], levels=8, grid_x=3, grid_y=3)
# gabor_filter_descriptor = GaborFilterDescriptor(frequencies=[0.1, 0.2, 0.3], orientations=[0, np.pi/4, np.pi/2, 3*np.pi/4])

features_dict = {}

for mode in modes:

    ## Data loader
    ### Limit the number of samples to 200 for training and load all samples for validation
    max_samples = None if mode == 'train' else None
    ### Balance the dataset for training
    balance = None if mode == 'train' else False
    dataloader = DataLoader(data_folder, mode,
                            shuffle=True,
                            ignore_folders=['black_background', '.DS_Store'],
                            max_samples=max_samples,
                            balance=balance,
                            transforms=None,
                            classes=CLASSES,
                            mask=False)

    ## Extract features
    features = []
    labels = []
    for i, (img, label, mask, path) in tqdm(enumerate(dataloader), total=len(dataloader), desc=f'Extracting features for {mode}'):
        color_features = color_descriptor.extract(img, mask=None)
        color_layout_features = color_layout_descriptor.extract(img, mask=None)
        intensity_stats_grid_features = intensity_stats_grid_descriptor.extract(img, mask=None)
        glcm_features, glcm_img = glcm_descriptor.extract(img, mask=None)
        lbp_features, lbp_img = lbp_descriptor.extract(img, mask=None)
        lbp2_features, lbp2_img = lbp2_descriptor.extract(img, mask=None)
        lbp3_features, lbp3_img = lbp3_descriptor.extract(img, mask=None)
        # color_cooccurrence_matrix_features = color_cooccurrence_matrix_descriptor.extract(img, mask=None)
        # gabors_features = gabor_filter_descriptor.extract(img, mask=None)
        features.append(np.concatenate([lbp_features, lbp2_features, lbp3_features, glcm_features, color_features, intensity_stats_grid_features], axis=0))

        ## add label
        labels.append(label)


    ## Save features to disk
    features = np.array(features)
    labels = np.array(labels)
    features_with_labels = np.concatenate([features, labels.reshape(-1, 1)], axis=1)
    features_dict[mode] = features_with_labels

## Training

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.metrics import cohen_kappa_score
from sklearn.preprocessing import StandardScaler
import xgboost as xgb

In [ ]:
mode = 'train'
# model = SVC(kernel='rbf', C=5.0, random_state=42, degree=5)
xgb_clf = xgb.XGBClassifier(objective='multi:softmax', num_class=3, n_estimators=3000, learning_rate=0.2, random_state=42)

In [ ]:
features = features_dict[mode][:, :-1]
labels = features_dict[mode][:, -1]

X_train, X_val, y_train, y_val = train_test_split(features, labels, test_size=0.2, random_state=42, stratify=labels)

In [ ]:
scaler = StandardScaler()
model = Pipeline([('scaler', scaler), ('model', xgb_clf)])

In [ ]:
model.fit(X_train, y_train)

Pipeline(steps=[('scaler', StandardScaler()),
                ('model',
                 XGBClassifier(base_score=None, booster=None, callbacks=None,
                               colsample_bylevel=None, colsample_bynode=None,
                               colsample_bytree=None, device=None,
                               early_stopping_rounds=None,
                               enable_categorical=False, eval_metric=None,
                               feature_types=None, gamma=None, grow_policy=None,
                               importance_type=None,
                               interaction_constraints=None, learning_rate=0.2,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=None, max_leaves=None,
                               min_child_weight=None, missing=nan,
                               monotone_constraints=None, multi_strategy=None,
                               n_estimators=3000, n_jobs=None, num_class=3,
                               num_parallel_tree=None, ...))])

## Validation

In [ ]:
y_pred = model.predict(X_val)
print(classification_report(y_val, y_pred, target_names=CLASSES))
print("Kappa score: ", cohen_kappa_score(y_val, y_pred))

              precision    recall  f1-score   support

         mel       0.91      0.94      0.92      1085
         bcc       0.89      0.85      0.87       797
         scc       0.91      0.91      0.91       602

    accuracy                           0.90      2484
   macro avg       0.90      0.90      0.90      2484
weighted avg       0.90      0.90      0.90      2484

Kappa score:  0.8504803409337154


## Testing

In [ ]:
mode = 'val'
features_test = features_dict[mode][:, :-1]
labels_test = features_dict[mode][:, -1]

y_pred = model.predict(features_test)
print(classification_report(labels_test, y_pred, target_names=CLASSES))
print("Kappa score: ", cohen_kappa_score(labels_test, y_pred))

              precision    recall  f1-score   support

         mel       0.91      0.90      0.91       678
         bcc       0.84      0.87      0.86       498
         scc       0.62      0.51      0.56        94

    accuracy                           0.86      1270
   macro avg       0.79      0.76      0.77      1270
weighted avg       0.86      0.86      0.86      1270

Kappa score:  0.7502438563921839


## Cross Validation

In [ ]:
from sklearn.model_selection import cross_val_score, cross_val_predict, StratifiedKFold

# Initialize XGBoost classifier
xgb_clf = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss', objective='multi:softmax', num_class=3, n_estimators=500, learning_rate=0.1, n_jobs=-1)

# Define cross-validation strategy (e.g., 5-fold stratified)
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Perform cross-validation and get accuracy scores for each fold
features_train = features_dict['train'][:, :-1]
labels_train = features_dict['train'][:, -1]
cv_scores = cross_val_score(xgb_clf, features_train, labels_train, cv=cv, scoring='accuracy')

# Output the results
print("Cross-Validation Accuracy Scores:", cv_scores)
print("Mean CV Accuracy:", cv_scores.mean())
print("Standard Deviation of CV Accuracy:", cv_scores.std())

Cross-Validation Accuracy Scores: [0.5 0.4 0.5 0.4 0.5]
Mean CV Accuracy: 0.45999999999999996
Standard Deviation of CV Accuracy: 0.04898979485566355


In [ ]:
# Cross-validation predictions (optional)
features_test = features_dict['val'][:, :-1]
labels_test = features_dict['val'][:, -1]
cv_predictions = cross_val_predict(xgb_clf, features_test, labels_test, cv=cv)
print("Classification Report for CV Predictions:\n", classification_report(labels_test, cv_predictions))

Classification Report for CV Predictions:
               precision    recall  f1-score   support

         0.0       0.70      0.76      0.73        25
         1.0       0.74      0.68      0.71        25

    accuracy                           0.72        50
   macro avg       0.72      0.72      0.72        50
weighted avg       0.72      0.72      0.72        50



## Grid Search

In [ ]:
from sklearn.model_selection import GridSearchCV

# Initialize the base model (XGBClassifier)
xgb_clf = xgb.XGBClassifier(objective='multi:softmax', num_class=3, n_jobs=-1, random_state=42)
scaler_cv = StandardScaler()

model = Pipeline([('scaler', scaler_cv), ('model', xgb_clf)])

# Define the parameter grid for Grid Search
param_grid = {
    'model__n_estimators': [1000, 2000],  # Number of trees
    'model__learning_rate': [0.1, 0.2],  # Step size shrinkage
    # 'reg_lambda': [1.0, 0.8],  # L2 regularization term on weights
}

# Initialize GridSearchCV
grid_search = GridSearchCV(estimator=model, param_grid=param_grid,
                        scoring='accuracy',  # Use 'accuracy' as the evaluation metric
                        cv=5,  # 5-fold cross-validation
                        verbose=4,
                        n_jobs=-1)  # Parallel processing

features_train = features_dict['train'][:, :-1]
labels_train = features_dict['train'][:, -1]
grid_search.fit(features_train, labels_train)

# Get the best parameters and best score from the grid search
print("Best Parameters: ", grid_search.best_params_)
print("Best Accuracy: ", grid_search.best_score_)

Fitting 5 folds for each of 4 candidates, totalling 20 fits
[CV 1/5] END model__learning_rate=0.1, model__n_estimators=1000;, score=0.911 total time=12.7min
[CV 5/5] END model__learning_rate=0.1, model__n_estimators=1000;, score=0.912 total time=12.7min
[CV 3/5] END model__learning_rate=0.1, model__n_estimators=1000;, score=0.897 total time=12.8min
[CV 4/5] END model__learning_rate=0.1, model__n_estimators=1000;, score=0.914 total time=12.8min
[CV 2/5] END model__learning_rate=0.1, model__n_estimators=1000;, score=0.913 total time=12.8min
[CV 3/5] END model__learning_rate=0.1, model__n_estimators=2000;, score=0.899 total time=17.1min
[CV 2/5] END model__learning_rate=0.1, model__n_estimators=2000;, score=0.913 total time=17.2min
[CV 1/5] END model__learning_rate=0.1, model__n_estimators=2000;, score=0.911 total time=17.3min
[CV 1/5] END model__learning_rate=0.2, model__n_estimators=1000;, score=0.913 total time= 5.5min
[CV 3/5] END model__learning_rate=0.2, model__n_estimators=1000;, s

In [ ]:
best_model = grid_search.best_estimator_
features_test = features_dict['val'][:, :-1]
labels_test = features_dict['val'][:, -1]

y_pred = best_model.predict(features_test)
print(classification_report(labels_test, y_pred, target_names=CLASSES))
print("Kappa score: ", cohen_kappa_score(labels_test, y_pred))

              precision    recall  f1-score   support

         mel       0.91      0.87      0.89       678
         bcc       0.82      0.87      0.85       498
         scc       0.52      0.53      0.53        94

    accuracy                           0.85      1270
   macro avg       0.75      0.76      0.76      1270
weighted avg       0.85      0.85      0.85      1270

Kappa score:  0.7269737279533335


## Exporting Experiment

In [ ]:
## Export experiment
notebook_name = 'MultiClass.ipynb'
export_experiment(name=exp_name, params=params, feature_dict=features_dict , model=best_model, notebook_name=notebook_name)

<IPython.core.display.Javascript object>

Experiment 'multi-class_classification' saved at experiments/multi-class_classification_20241104_022653


## Combine all data for testing

In [ ]:
train_features = np.load('/Users/madzie/Workspace/UdG/CAD/Skin-Lesion-Classification1/experiments/Best/train.npy')
val_features = np.load('/Users/madzie/Workspace/UdG/CAD/Skin-Lesion-Classification1/experiments/Best/val.npy')

In [ ]:
all_features = np.concatenate([train_features[:, :-1], val_features[:, :-1]], axis=0)
all_labels = np.concatenate([train_features[:, -1], val_features[:, -1]], axis=0)

pipeline = Pipeline([('scaler', scaler_cv), ('model', xgb_clf)])
pipeline.fit(all_features, all_labels)

Pipeline(steps=[('scaler', StandardScaler()),
                ('model',
                 XGBClassifier(base_score=None, booster=None, callbacks=None,
                               colsample_bylevel=None, colsample_bynode=None,
                               colsample_bytree=None, device=None,
                               early_stopping_rounds=None,
                               enable_categorical=False, eval_metric=None,
                               feature_types=None, gamma=None, grow_policy=None,
                               importance_type=None,
                               interaction_constraints=None, learning_rate=0.2,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=None, max_leaves=None,
                               min_child_weight=None, missing=nan,
                               monotone_constraints=None, multi_strategy=None,
                               n_estimators=3000, n_jobs=None, num_class=3,
                               num_parallel_tree=None, ...))])

## Predict on test set

In [ ]:
test_folder = os.path.join(data_folder, 'test')

indices = []
results = []

### Load test images
image_files = sorted(os.listdir(test_folder))
for img_name in tqdm(image_files, desc='Predicting test images ...'):
    image_idx = int(img_name.split('.')[0].replace('xxx', ''))
    indices.append(image_idx)

    ## Load image
    img_path = os.path.join(test_folder, img_name)
    img = cv2.imread(img_path)

    ## Extract features
    color_features = color_descriptor.extract(img, mask=None)
    color_layout_features = color_layout_descriptor.extract(img, mask=None)
    intensity_stats_grid_features = intensity_stats_grid_descriptor.extract(img, mask=None)
    glcm_features, glcm_img = glcm_descriptor.extract(img, mask=None)
    lbp_features, lbp_img = lbp_descriptor.extract(img, mask=None)
    lbp2_features, lbp2_img = lbp2_descriptor.extract(img, mask=None)
    lbp3_features, lbp3_img = lbp3_descriptor.extract(img, mask=None)

    ## Concatenate features
    features = np.concatenate([lbp_features, lbp2_features, lbp3_features, glcm_features, color_features, intensity_stats_grid_features], axis=0)
    features = features.reshape(1, -1)

    ## Predict
    y_pred = pipeline.predict(features)
    results.append(y_pred[0])

Predicting test images ...: 100%|██████████| 2121/2121 [11:44<00:00,  3.01it/s]


In [ ]:
## Export results to CSV
import csv

csv_path = 'challenge2.csv'

with open(csv_path, mode='w') as file:
    writer = csv.writer(file)
    for result in results:
        writer.writerow([result])

In [ ]:
import pandas as pd
df = pd.read_csv('challenge2_single.csv', header=None)
df_new = pd.read_csv('challenge2.csv', header=None)
old_results = df[0].values
new_results = df_new[0].values

In [ ]:
# Count the number of indices with the same values in both lists
overlap_count = sum(1 for a, b in zip(old_results, new_results) if a == b)

print("Number of matching indices:", overlap_count/len(old_results))

Number of matching indices: 0.9552098066949553


In [ ]:
np.unique(results, return_counts=True)

(array([0, 1, 2], dtype=int32), array([1127,  884,  110]))